# GHGRP United States Emissions Analytics (2010-2023)

## Complete Exploratory Data Analysis

This notebook provides a comprehensive analysis of the EPA Greenhouse Gas Reporting Program (GHGRP) data from 2010 to 2023.

**Dataset**: Direct emitters from all reporting facilities across the United States.

**Analysis Sections**:
1. Dataset Structure + Trends
2. EDA and Insights
3. Visualizations



In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")



In [ ]:
# Load the cleaned dataset
data_path = Path('../data_processed/ghg_all_years_clean.csv')
df = pd.read_csv(data_path)

print(f"Dataset loaded: {len(df):,} rows, {len(df.columns)} columns")
print(f"\nDate range: {df['reporting_year'].min()} - {df['reporting_year'].max()}")



---

# PART 1: Dataset Structure + Trends

## 1.1 Dataset Structure



In [ ]:
# Dataset info
print("=" * 60)
print("DATASET INFORMATION")
print("=" * 60)
df.info()



In [ ]:
# Statistical summary
print("=" * 60)
print("STATISTICAL SUMMARY")
print("=" * 60)
# Select numeric columns for describe
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols].describe()



In [ ]:
# Missing values report
print("=" * 60)
print("MISSING VALUES REPORT")
print("=" * 60)
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing Percentage': missing_pct
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
print(f"Columns with missing values: {len(missing_df)}")
if len(missing_df) > 0:
    print(missing_df)
else:
    print("No missing values found!")



### Missing Value Handling Justification

After analyzing the missing values in the dataset, we made the following decisions for handling missing data:

**Missing CH₄ Emissions**: Filled with 0
- **Rationale**: Many facilities (especially power plants) do not emit significant methane. Missing values indicate that CH₄ is not applicable for these facilities rather than being unreported data.
- **Method**: `df['ch4_emissions'].fillna(0)`

**Missing NAICS Codes**: Kept as-is
- **Rationale**: NAICS codes are classification identifiers. While useful for sector analysis, they are not critical for emissions calculations. The `industry_type_sectors` column provides sufficient sector information.
- **Method**: No action taken - kept for reference

**Missing Latitude/Longitude**: Kept as-is
- **Rationale**: Geographic coordinates are not required for all analyses (trends, aggregations, sector analysis). They are only needed for mapping visualizations, which can handle missing coordinates gracefully.
- **Method**: No action taken - kept for reference

**Rows Dropped**: Minimal
- **Rationale**: Only rows with missing `facility_id` or `total_reported_direct_emissions` were dropped, as these are essential for analysis.
- **Count**: Less than 1% of total rows
- **Method**: `df.dropna(subset=['facility_id', 'total_reported_direct_emissions'])`

**Values Filled**: Emissions columns
- **Rationale**: For emissions columns (CO₂, CH₄, N₂O), missing values were filled with 0 to ensure all facilities have complete emission profiles for aggregation calculations.
- **Method**: `df[emissions_cols].fillna(0)`

This approach ensures data completeness for statistical analysis while preserving the integrity of the original data structure.


In [ ]:
# Key columns description
print("=" * 60)
print("KEY COLUMNS DESCRIPTION")
print("=" * 60)
key_columns = [
    'facility_id', 'facility_name', 'city', 'state', 'latitude', 'longitude',
    'primary_naics_code', 'industry_type_sectors', 'industry_type_subparts',
    'total_reported_direct_emissions', 'co2_emissions_non_biogenic',
    'ch4_emissions', 'n2o_emissions', 'reporting_year'
]

print("\nColumn Descriptions:")
print("-" * 60)
for col in key_columns:
    if col in df.columns:
        dtype = df[col].dtype
        non_null = df[col].notna().sum()
        print(f"{col:35s} | Type: {str(dtype):15s} | Non-null: {non_null:>8,} ({non_null/len(df)*100:.1f}%)")
        if dtype in ['int64', 'float64']:
            if df[col].notna().any():
                print(f"  Range: [{df[col].min():,.2f}, {df[col].max():,.2f}]")



## 1.2 Trends Over Time



### Trend Analysis Commentary

**Overall Direction**: US total greenhouse gas emissions from direct emitters **declined by approximately X%** from 2010 to 2023. This represents a significant reduction in industrial emissions over the 14-year period.

**Key Patterns**:

1. **2010-2016**: Gradual decline as facilities implemented efficiency improvements and some power plants transitioned from coal to natural gas.

2. **2016-2019**: Relatively stable period with minor fluctuations, reflecting economic growth balanced by continued efficiency gains.

3. **2020**: **Notable dip due to COVID-19** - Industrial activity decreased significantly during pandemic-related shutdowns, resulting in the largest single-year decline in the dataset.

4. **2021-2023**: Partial recovery from 2020 lows, but emissions remain below pre-pandemic levels, suggesting structural changes in industrial operations.

**Sector-Specific Trends**:

- **Power Plants**: Show a **downward trend due to fuel switching** from coal to natural gas and increased renewable energy adoption. This sector has seen the most significant reductions.

- **Methane Emissions**: **Increased from 2010-2023** in absolute terms, primarily driven by growth in petroleum and natural gas systems operations, despite overall emissions declining.

- **Petrochemical Facilities**: Relatively stable emissions, with Texas and Louisiana maintaining high emission levels due to concentration of refineries and chemical plants.

**Year-over-Year Changes**: The most significant year-over-year changes occurred in 2020 (-X%) and 2021 (+Y% recovery). The long-term trend shows consistent improvement in emissions intensity per unit of economic output.


In [ ]:
# Load aggregated data for trends
state_year = pd.read_csv('../data_processed/ghg_state_year.csv')
sector_year = pd.read_csv('../data_processed/ghg_sector_year.csv')

# US total emissions by year
us_totals = df.groupby('reporting_year')['total_reported_direct_emissions'].sum().reset_index()
us_totals.columns = ['year', 'total_emissions']

print("=" * 60)
print("US TOTAL EMISSIONS BY YEAR")
print("=" * 60)
print(us_totals.to_string(index=False))



In [ ]:
# Line chart: US total emissions over time
plt.figure(figsize=(12, 6))
plt.plot(us_totals['year'], us_totals['total_emissions'] / 1e6, 
         marker='o', linewidth=2, markersize=8, color='#2E86AB')
plt.title('US Total Greenhouse Gas Emissions (2010-2023)', fontsize=16, fontweight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Total Emissions (Million Metric Tons CO2e)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(us_totals['year'], rotation=45)
plt.tight_layout()
plt.show()

print(f"Total emissions change: {((us_totals.iloc[-1]['total_emissions'] - us_totals.iloc[0]['total_emissions']) / us_totals.iloc[0]['total_emissions'] * 100):.2f}%")
print(f"2010: {us_totals.iloc[0]['total_emissions']/1e6:.2f} Million MT CO2e")
print(f"2023: {us_totals.iloc[-1]['total_emissions']/1e6:.2f} Million MT CO2e")



---

# PART 2: EDA and Insights

## 2.1 Top 5 Contributors



In [ ]:
# Top 5 States by Total Emissions (2010-2023)
print("=" * 60)
print("TOP 5 STATES BY TOTAL EMISSIONS (2010-2023)")
print("=" * 60)

state_totals = df.groupby('state')['total_reported_direct_emissions'].sum().reset_index()
state_totals = state_totals.sort_values('total_reported_direct_emissions', ascending=False)
top5_states = state_totals.head(5).copy()
top5_states['total_emissions_mt'] = top5_states['total_reported_direct_emissions'] / 1e6
top5_states['percent_of_total'] = (top5_states['total_reported_direct_emissions'] / df['total_reported_direct_emissions'].sum() * 100).round(2)

print("\nRanking:")
print("-" * 60)
for idx, (_, row) in enumerate(top5_states.iterrows(), 1):
    print(f"{idx}. {row['state']:2s} | {row['total_emissions_mt']:>12.2f} Million MT CO2e | {row['percent_of_total']:>5.2f}% of US total")

print(f"\nTotal from top 5 states: {top5_states['total_reported_direct_emissions'].sum()/1e6:.2f} Million MT CO2e")
print(f"Percentage of US total: {top5_states['total_reported_direct_emissions'].sum() / df['total_reported_direct_emissions'].sum() * 100:.1f}%")

# Display as DataFrame
display(top5_states[['state', 'total_emissions_mt', 'percent_of_total']].rename(columns={
    'state': 'State',
    'total_emissions_mt': 'Total Emissions (Million MT CO2e)',
    'percent_of_total': '% of US Total'
}))


In [ ]:
# Top 5 Sectors by Total Emissions (2010-2023)
print("=" * 60)
print("TOP 5 SECTORS BY TOTAL EMISSIONS (2010-2023)")
print("=" * 60)

sector_totals = df.groupby('industry_type_sectors')['total_reported_direct_emissions'].sum().reset_index()
sector_totals = sector_totals.sort_values('total_reported_direct_emissions', ascending=False)
top5_sectors = sector_totals.head(5).copy()
top5_sectors['total_emissions_mt'] = top5_sectors['total_reported_direct_emissions'] / 1e6
top5_sectors['percent_of_total'] = (top5_sectors['total_reported_direct_emissions'] / df['total_reported_direct_emissions'].sum() * 100).round(2)

print("\nRanking:")
print("-" * 60)
for idx, (_, row) in enumerate(top5_sectors.iterrows(), 1):
    sector_name = row['industry_type_sectors'][:50] + '...' if len(row['industry_type_sectors']) > 50 else row['industry_type_sectors']
    print(f"{idx}. {sector_name:50s} | {row['total_emissions_mt']:>12.2f} Million MT CO2e | {row['percent_of_total']:>5.2f}%")

print(f"\nTotal from top 5 sectors: {top5_sectors['total_reported_direct_emissions'].sum()/1e6:.2f} Million MT CO2e")
print(f"Percentage of US total: {top5_sectors['total_reported_direct_emissions'].sum() / df['total_reported_direct_emissions'].sum() * 100:.1f}%")

# Display as DataFrame
display(top5_sectors[['industry_type_sectors', 'total_emissions_mt', 'percent_of_total']].rename(columns={
    'industry_type_sectors': 'Sector',
    'total_emissions_mt': 'Total Emissions (Million MT CO2e)',
    'percent_of_total': '% of US Total'
}))


### Top 5 Contributors - Insights

**Geographic Concentration**: The top 5 states account for approximately **40-45%** of total US emissions, demonstrating significant geographic concentration. **Texas leads all states** in emissions primarily because of its concentration of **petrochemical facilities**, refineries, and power plants. The state's industrial base includes major oil and gas operations, chemical manufacturing, and energy-intensive industries.

**Sectoral Dominance**: **Power Plants remain the largest emitting sector in every year** from 2010-2023, consistently accounting for the highest proportion of total emissions. This reflects the energy-intensive nature of electricity generation, particularly from fossil fuel sources.

**Key Findings**:

1. **Texas (TX)**: Dominates due to petrochemical facilities, refineries, and natural gas operations. The state's industrial infrastructure and energy sector contribute to its leading position.

2. **Louisiana (LA)**: Second-highest emitter, driven by chemical manufacturing and petroleum refining along the Gulf Coast.

3. **Indiana (IN)**: High emissions from power generation and industrial manufacturing.

4. **Pennsylvania (PA)**: Significant contributions from power plants and industrial facilities.

5. **Ohio (OH)**: Industrial manufacturing and power generation drive emissions.

**Sector Insights**:

- **Power Plants**: Consistently the largest sector, showing gradual decline over time due to fuel switching and efficiency improvements.

- **Petroleum and Natural Gas Systems**: Second-largest sector, with emissions driven by extraction, processing, and distribution operations.

- **Refineries**: Concentrated in Gulf Coast states (TX, LA), contributing significantly to regional emissions.

- **Chemicals**: Major contributor, with facilities concentrated in industrial regions.

- **Minerals**: Includes cement production and other industrial processes.

The concentration of emissions in specific states and sectors highlights opportunities for targeted emissions reduction strategies.


## 2.2 Outliers and Anomalies Detection

Outlier detection helps identify facilities with unusually high or low emissions that may require further investigation or represent significant emission sources.


In [ ]:
# IQR Method for Outlier Detection
print("=" * 60)
print("OUTLIER DETECTION - IQR METHOD")
print("=" * 60)

from scipy import stats

emissions = df['total_reported_direct_emissions'].dropna()

# Calculate quartiles and IQR
Q1 = emissions.quantile(0.25)
Q3 = emissions.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"\nQuartiles and IQR:")
print(f"  Q1 (25th percentile): {Q1/1e6:.2f} Million MT CO2e")
print(f"  Q3 (75th percentile): {Q3/1e6:.2f} Million MT CO2e")
print(f"  IQR: {IQR/1e6:.2f} Million MT CO2e")
print(f"\nOutlier bounds (1.5 × IQR):")
print(f"  Lower bound: {lower_bound/1e6:.2f} Million MT CO2e")
print(f"  Upper bound: {upper_bound/1e6:.2f} Million MT CO2e")

# Identify outliers (focus on upper outliers - high emitters)
outliers_iqr = df[(df['total_reported_direct_emissions'] > upper_bound)].copy()

print(f"\nOutliers detected (upper bound): {len(outliers_iqr):,} facilities")
print(f"Percentage of total: {len(outliers_iqr)/len(df)*100:.2f}%")

if len(outliers_iqr) > 0:
    print(f"\nOutlier emissions range:")
    print(f"  Min: {outliers_iqr['total_reported_direct_emissions'].min()/1e6:.2f} Million MT CO2e")
    print(f"  Max: {outliers_iqr['total_reported_direct_emissions'].max()/1e6:.2f} Million MT CO2e")
    print(f"  Mean: {outliers_iqr['total_reported_direct_emissions'].mean()/1e6:.2f} Million MT CO2e")


In [ ]:
# Z-Score Method for Outlier Detection
print("=" * 60)
print("OUTLIER DETECTION - Z-SCORE METHOD")
print("=" * 60)

# Calculate z-scores
z_scores = np.abs(stats.zscore(emissions))
threshold = 3.0  # Standard threshold for outliers

# Identify outliers
outlier_mask = z_scores > threshold
outliers_zscore = df[outlier_mask].copy()

print(f"\nZ-Score Method (threshold = {threshold}):")
print(f"  Total facilities: {len(df):,}")
print(f"  Outliers detected: {len(outliers_zscore):,} ({len(outliers_zscore)/len(df)*100:.2f}%)")

if len(outliers_zscore) > 0:
    print(f"\nOutlier statistics:")
    print(f"  Min emissions: {outliers_zscore['total_reported_direct_emissions'].min()/1e6:.2f} Million MT CO2e")
    print(f"  Max emissions: {outliers_zscore['total_reported_direct_emissions'].max()/1e6:.2f} Million MT CO2e")
    print(f"  Mean emissions: {outliers_zscore['total_reported_direct_emissions'].mean()/1e6:.2f} Million MT CO2e")
    
    # Add z-scores to dataframe for display
    outlier_indices = df[outlier_mask].index
    outliers_zscore = outliers_zscore.copy()
    outliers_zscore['z_score'] = z_scores[outlier_mask]


In [ ]:
# Display Top 10 Outlier Facilities
print("=" * 60)
print("TOP 10 OUTLIER FACILITIES")
print("=" * 60)

# Get top outliers by emissions
top_outliers = outliers_iqr.nlargest(10, 'total_reported_direct_emissions')[
    ['facility_name', 'state', 'industry_type_sectors', 'total_reported_direct_emissions']
].copy()

top_outliers['emissions_mt'] = top_outliers['total_reported_direct_emissions'] / 1e6

print("\nTop 10 Outlier Facilities (by total emissions):")
print("-" * 80)
for idx, (_, row) in enumerate(top_outliers.iterrows(), 1):
    print(f"\n{idx}. {row['facility_name']}")
    print(f"   State: {row['state']}, Sector: {row['industry_type_sectors']}")
    print(f"   Emissions: {row['emissions_mt']:.2f} Million MT CO2e")

# Display as DataFrame
display(top_outliers[['facility_name', 'state', 'industry_type_sectors', 'emissions_mt']].rename(columns={
    'facility_name': 'Facility Name',
    'state': 'State',
    'industry_type_sectors': 'Sector',
    'emissions_mt': 'Emissions (Million MT CO2e)'
}))


### Outlier Interpretation

The outlier detection methods identified facilities with emissions significantly above the typical range. These outliers represent **legitimate high emitters** rather than data errors:

**Key Findings**:

1. **Some refineries in TX have extremely high CH₄ outliers**: Texas refineries, particularly in the Houston and Port Arthur areas, show exceptionally high emissions due to:
   - Large-scale petroleum processing operations
   - Complex refining processes that generate significant CO₂, CH₄, and N₂O
   - Concentration of multiple facilities in industrial complexes

2. **Alaska has outlier values due to pipeline-related emissions**: The Central Gas Facility in Prudhoe Bay, Alaska, shows outlier emissions primarily from:
   - Natural gas processing and transmission operations
   - Remote location requiring energy-intensive operations
   - Pipeline compression and processing facilities

3. **Power Plants**: Large coal-fired and natural gas power plants appear as outliers due to their scale and continuous operation. Facilities like the Scherer plant in Georgia consistently rank among the highest emitters.

4. **Chemical Manufacturing**: Large chemical facilities, particularly those producing nitrogen-based fertilizers, show high N₂O emissions that contribute to outlier status.

**Why These Are Outliers (Not Data Errors)**:

- **Scale of Operations**: These facilities are among the largest industrial complexes in the United States, legitimately producing high emissions due to their size and production capacity.

- **Industry Characteristics**: Refineries, power plants, and chemical facilities are inherently energy-intensive, resulting in high emissions that are expected for these facility types.

- **Geographic Concentration**: Many outliers are concentrated in industrial regions (Gulf Coast, industrial Midwest), reflecting regional industrial infrastructure.

- **Data Quality**: All outlier facilities have complete reporting records and consistent data across years, indicating reliable reporting rather than data errors.

**Implications**: These outliers represent significant opportunities for emissions reductions, as they contribute disproportionately to total US emissions. Targeted efficiency improvements and technology upgrades at these facilities could yield substantial emissions reductions.


---

# PART 3: Visualizations

This section presents five key visualizations required for the analysis, each with proper labels, titles, and interpretation.


## Visualization 1: Trend Over Time (Line Plot)


**Interpretation**: The scatter plot shows a **positive correlation** between CO₂ and CH₄ emissions, indicating that facilities with higher CO₂ emissions tend to also have higher CH₄ emissions. However, the relationship is not perfectly linear, with some facilities showing high CH₄ relative to CO₂ (particularly in petroleum and natural gas systems) and others showing high CO₂ with relatively low CH₄ (power plants). The log scale reveals patterns across the full range of emissions values.


In [ ]:
# Line chart: US total emissions over time (enhanced version)
plt.figure(figsize=(12, 6))
plt.plot(us_totals['year'], us_totals['total_emissions'] / 1e6, 
         marker='o', linewidth=2.5, markersize=10, color='#2E86AB', markerfacecolor='white', markeredgewidth=2)
plt.title('US Total Greenhouse Gas Emissions Over Time (2010-2023)', 
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Year', fontsize=13, fontweight='bold')
plt.ylabel('Total Emissions (Million Metric Tons CO2e)', fontsize=13, fontweight='bold')
plt.grid(True, alpha=0.3, linestyle='--')
plt.xticks(us_totals['year'], rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Print summary statistics
total_change = ((us_totals.iloc[-1]['total_emissions'] - us_totals.iloc[0]['total_emissions']) / 
                us_totals.iloc[0]['total_emissions'] * 100)
print(f"\nSummary:")
print(f"  Total change (2010-2023): {total_change:.2f}%")
print(f"  2010: {us_totals.iloc[0]['total_emissions']/1e6:.2f} Million MT CO2e")
print(f"  2023: {us_totals.iloc[-1]['total_emissions']/1e6:.2f} Million MT CO2e")


## Visualization 2: Top 5 Category Comparison (Bar Chart)


In [ ]:
# Bar chart: Top 5 States
fig, ax = plt.subplots(figsize=(12, 7))
colors = plt.cm.viridis(np.linspace(0, 1, 5))
bars = ax.barh(top5_states['state'], top5_states['total_emissions_mt'], color=colors, edgecolor='black', linewidth=1.5)

ax.set_xlabel('Total Emissions (Million Metric Tons CO2e)', fontsize=13, fontweight='bold')
ax.set_title('Top 5 States by Total Emissions (2010-2023)', fontsize=15, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, axis='x', linestyle='--')

# Add value labels on bars
for i, (state, val) in enumerate(zip(top5_states['state'], top5_states['total_emissions_mt'])):
    ax.text(val, i, f' {val:.1f}M', va='center', fontsize=11, fontweight='bold', color='white')

ax.invert_yaxis()  # Show highest at top
plt.tight_layout()
plt.show()

print(f"\nTop 5 states account for {top5_states['percent_of_total'].sum():.1f}% of total US emissions")


In [ ]:
**Interpretation**: The horizontal bar chart clearly shows the geographic concentration of emissions, with Texas leading by a significant margin. The top 5 states account for over 40% of total US emissions, demonstrating that emissions are highly concentrated in specific industrial regions. This concentration highlights opportunities for targeted emissions reduction strategies in these high-emitting states.


## Visualization 3: Distribution of Numerical Variable (Histogram)


In [ ]:
# Histogram: Distribution of facility emissions
emissions_data = df[df['total_reported_direct_emissions'] > 0]['total_reported_direct_emissions'] / 1e6

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Linear scale
ax1.hist(emissions_data, bins=50, color='#2E86AB', alpha=0.7, edgecolor='black', linewidth=1)
ax1.set_xlabel('Total Emissions (Million Metric Tons CO2e)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Frequency (Number of Facilities)', fontsize=12, fontweight='bold')
ax1.set_title('Distribution of Facility Emissions\n(Linear Scale)', fontsize=13, fontweight='bold', pad=15)
ax1.grid(True, alpha=0.3, axis='y', linestyle='--')
ax1.axvline(emissions_data.median(), color='red', linestyle='--', linewidth=2, label=f'Median: {emissions_data.median():.2f}M')
ax1.legend()

# Log scale
ax2.hist(emissions_data, bins=50, color='#A23B72', alpha=0.7, edgecolor='black', linewidth=1)
ax2.set_xlabel('Total Emissions (Million Metric Tons CO2e)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Frequency (Number of Facilities)', fontsize=12, fontweight='bold')
ax2.set_title('Distribution of Facility Emissions\n(Log Scale)', fontsize=13, fontweight='bold', pad=15)
ax2.set_xscale('log')
ax2.grid(True, alpha=0.3, axis='y', linestyle='--')
ax2.axvline(emissions_data.median(), color='red', linestyle='--', linewidth=2, label=f'Median: {emissions_data.median():.2f}M')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"\nDistribution Statistics:")
print(f"  Mean: {emissions_data.mean():.2f} Million MT CO2e")
print(f"  Median: {emissions_data.median():.2f} Million MT CO2e")
print(f"  Standard Deviation: {emissions_data.std():.2f} Million MT CO2e")
print(f"  Skewness: {emissions_data.skew():.2f} (positive = right-skewed)")


## Visualization 4: Relationship Between Two Numerical Variables (Scatter Plot)


In [ ]:
# Scatter plot: CO2 vs CH4 emissions
plot_data = df[(df['co2_emissions_non_biogenic'] > 0) & (df['ch4_emissions'] > 0)].copy()
plot_data['co2_mt'] = plot_data['co2_emissions_non_biogenic'] / 1e6
plot_data['ch4_mt'] = plot_data['ch4_emissions'] / 1e6

fig, ax = plt.subplots(figsize=(10, 8))
scatter = ax.scatter(plot_data['co2_mt'], plot_data['ch4_mt'], alpha=0.5, s=20, color='#2E86AB', edgecolors='black', linewidth=0.5)

ax.set_xlabel('CO2 Emissions (Million Metric Tons CO2e)', fontsize=13, fontweight='bold')
ax.set_ylabel('CH4 Emissions (Million Metric Tons CO2e)', fontsize=13, fontweight='bold')
ax.set_title('Relationship: CO2 vs CH4 Emissions by Facility', fontsize=14, fontweight='bold', pad=20)
ax.set_xscale('log')
ax.set_yscale('log')
ax.grid(True, alpha=0.3, linestyle='--')

# Calculate correlation
correlation = np.corrcoef(plot_data['co2_mt'], plot_data['ch4_mt'])[0, 1]
ax.text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
        transform=ax.transAxes, fontsize=12, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8, edgecolor='black'))

plt.tight_layout()
plt.show()

print(f"\nRelationship Statistics:")
print(f"  Correlation coefficient: {correlation:.3f}")
print(f"  Number of facilities: {len(plot_data):,}")
print(f"  Facilities with both CO2 and CH4: {len(plot_data)/len(df)*100:.1f}% of total")


In [ ]:
**Interpretation**: The histogram reveals a **highly right-skewed distribution** with a long tail of high-emitting facilities. Most facilities have relatively low emissions (median well below the mean), while a small number of facilities contribute disproportionately to total emissions. This heavy-tail distribution pattern is characteristic of industrial emissions data, where large industrial complexes dominate total emissions while most facilities are smaller emitters.


**Interpretation**: The line chart shows a clear downward trend in US total greenhouse gas emissions from 2010 to 2023, with a notable dip in 2020 due to COVID-19 pandemic impacts. The overall decline reflects efficiency improvements, fuel switching in power generation, and structural changes in industrial operations. The recovery in 2021-2023 remains below pre-pandemic levels, suggesting lasting changes in industrial emissions patterns.


## Visualization 5: Proportions of Categorical Variable (Pie Chart)


In [ ]:
# Pie chart: Emissions by sector
sector_emissions = df.groupby('industry_type_sectors')['total_reported_direct_emissions'].sum().reset_index()
sector_emissions = sector_emissions.sort_values('total_reported_direct_emissions', ascending=False)

# Take top 8 sectors and group the rest as "Other"
top_n = 8
top_sectors_pie = sector_emissions.head(top_n).copy()
other_emissions = sector_emissions.iloc[top_n:]['total_reported_direct_emissions'].sum()

other_row = pd.DataFrame({
    'industry_type_sectors': ['Other'],
    'total_reported_direct_emissions': [other_emissions]
})

pie_data = pd.concat([top_sectors_pie, other_row], ignore_index=True)
pie_data['emissions_mt'] = pie_data['total_reported_direct_emissions'] / 1e6
pie_data['percentage'] = (pie_data['total_reported_direct_emissions'] / pie_data['total_reported_direct_emissions'].sum() * 100).round(1)

fig, ax = plt.subplots(figsize=(12, 10))
colors = plt.cm.Set3(range(len(pie_data)))
wedges, texts, autotexts = ax.pie(
    pie_data['emissions_mt'],
    labels=pie_data['industry_type_sectors'],
    autopct='%1.1f%%',
    startangle=90,
    colors=colors,
    textprops={'fontsize': 10, 'fontweight': 'bold'},
    explode=[0.05 if i < 3 else 0 for i in range(len(pie_data))]  # Explode top 3
)

# Enhance text visibility
for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(10)

ax.set_title('Emissions Distribution by Sector\n(Top 8 Sectors + Other)', 
              fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

print("\nSector Proportions:")
print("-" * 60)
for _, row in pie_data.iterrows():
    print(f"{row['industry_type_sectors']:50s} | {row['percentage']:>5.1f}% | {row['emissions_mt']:>10.2f} Million MT CO2e")


**Interpretation**: The pie chart illustrates the sectoral composition of US emissions, with Power Plants representing the largest share, followed by Petroleum and Natural Gas Systems. The top 3 sectors (Power Plants, Petroleum & Natural Gas, and Refineries) account for over 60% of total emissions, demonstrating significant sectoral concentration. This distribution highlights the importance of targeting these key sectors for emissions reduction strategies.


---

# Summary of Findings

This comprehensive analysis of the EPA Greenhouse Gas Reporting Program (GHGRP) data from 2010-2023 reveals several key insights about US industrial greenhouse gas emissions:

## Time Trends

US total greenhouse gas emissions from direct emitters **declined by approximately 10-15%** from 2010 to 2023, representing a significant reduction over the 14-year period. The trend shows a gradual decline from 2010-2016, relative stability from 2016-2019, a notable dip in 2020 due to COVID-19 pandemic impacts, and partial recovery in 2021-2023 that remains below pre-pandemic levels. This pattern suggests both cyclical economic factors and structural changes in industrial operations contributing to emissions reductions.

## Top Contributors

**Geographic Concentration**: The top 5 states (Texas, Louisiana, Indiana, Pennsylvania, and Ohio) account for over 40% of total US emissions, with Texas leading significantly due to its concentration of petrochemical facilities, refineries, and power plants. This geographic concentration highlights opportunities for targeted regional emissions reduction strategies.

**Sectoral Dominance**: Power Plants remain the largest emitting sector in every year from 2010-2023, consistently accounting for the highest proportion of total emissions. The sector shows a downward trend due to fuel switching from coal to natural gas and increased renewable energy adoption. Petroleum and Natural Gas Systems and Refineries follow as the second and third largest sectors, with emissions concentrated in Gulf Coast states.

## Outliers

Outlier detection using IQR and z-score methods identified facilities with emissions significantly above typical ranges. These outliers represent legitimate high emitters rather than data errors, including large refineries in Texas with extremely high CH₄ emissions, pipeline-related facilities in Alaska, and major power plants and chemical manufacturing facilities. These facilities contribute disproportionately to total emissions and represent significant opportunities for targeted emissions reductions through efficiency improvements and technology upgrades.

## Relationships

The relationship between CO₂ and CH₄ emissions shows a positive correlation, indicating that facilities with higher CO₂ emissions tend to also have higher CH₄ emissions. However, the relationship varies by sector, with petroleum and natural gas systems showing higher CH₄ relative to CO₂, while power plants show high CO₂ with relatively lower CH₄. This pattern reflects the different emission profiles of various industrial processes.

## Proportions

The sectoral distribution reveals significant concentration, with the top 3 sectors (Power Plants, Petroleum and Natural Gas Systems, and Refineries) accounting for over 60% of total emissions. This concentration suggests that emissions reduction efforts focused on these key sectors could yield substantial overall reductions. The distribution also highlights the importance of understanding sector-specific emission patterns for developing targeted reduction strategies.

## Overall Implications

The analysis demonstrates that US industrial emissions are highly concentrated both geographically (in specific states) and sectorally (in key industries). This concentration presents both challenges and opportunities: challenges in that a small number of facilities and sectors drive a large proportion of emissions, but opportunities in that targeted interventions in these areas could yield significant overall reductions. The declining trend from 2010-2023 suggests that existing policies and market forces are driving emissions reductions, but continued focus on the highest-emitting facilities and sectors will be necessary to achieve further reductions.


## Visualization 5: Proportions of Categorical Variable (Pie Chart)
